In [ ]:
import pandas as pd
mdf = pd.read_csv('data\melb_data_fe.csv', sep=',')
data = mdf.copy()
# Приведение даты в формат datetime
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
# Выделение кварталов из дат и подсчет элементов в каждом квартале
sec = data['Date'].dt.quarter.value_counts()
# Вывод второго по популярности элемента
sec.loc[2]

In [ ]:
# Столбцы исключенные из преобразования форматов
cols_to_exclude = ['Date', 'Bedroom', 'Rooms', 'Bathroom', 'Car']

# Фильтрационный предел по Количеству Уникальных Элементов в столбце
max_unique_count = 150

# Перебор столбцов в таблице
for col in data.columns:
    
    # Фильтрация по К.У.Э. и по присутствию столбцов в списке искючений
    if data[col].nunique() < max_unique_count and col not in cols_to_exclude:
        
        # Перевод отфильтрованных столбцов в нужный формат
        data[col] = data[col].astype('category')

data.info()

In [ ]:
# Сортировка по Цене (По Возрастающей)
#data.sort_values(by='Price').head(10)

# Сортировка по Дате (По Убывающей)
#data.sort_values(by='Date', ascending=False)

# Сортировка по Удаленности от ЦГ и Цене (По Возрастающей)
data.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]# 


In [ ]:
mask2 = data['Type'] == 'townhouse'
mask1 = data['AreaRatio'] < -0.8 # Коэфф. соотнош. площадей здания и участка
mask3 = data['SellerG'] == 'McGrath'
data[mask1 & mask2 & mask3].sort_values( # Сортировка значений по маскам
    by=['Date', 'AreaRatio'], ascending=[True, False], ignore_index=True # Восх. по Датам. 
    ).loc[:, ['Date', 'AreaRatio']] # Отображение Дат и Коэфф.

In [ ]:
dc = data.copy()

# Сортировка Коэфф. по убывающей с заменой индексовб и выделение 1558 строки в столбце BuildingArea
dc.sort_values(by='AreaRatio', ascending=False, ignore_index=True).loc[1558, 'BuildingArea']

In [ ]:
town = dc['Type'] == 'townhouse' # Маска типа объектов
rooms = dc['Rooms'] > 2 # Маска кол-ва комнат 
dc[rooms & town].sort_values( 
    by=['Rooms', 'MeanRoomsSquare'], ascending=[True, False] #
    , ignore_index=True).loc[18, 'Price']#

In [ ]:
# Группировка столбца Regionname по Удаленности (Distance) по убывающей
data.groupby(by='Regionname')['Distance'].min().sort_values(ascending=False) 

In [ ]:
# Группировка столбца MonthSale по Цене (Price) по убывающей
data.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

In [ ]:
# Группировка столбца Regionname по количеству уникальных продавцов
data.groupby(by='Regionname')['SellerG'].agg(['nunique', set])

In [ ]:
# Группировка по количеству Комнат с самой высокой Ценой по Убывающей
data.groupby(by='Rooms')['Price'].mean().sort_values(ascending=False)

In [ ]:
# Группировка Регионов по Стандартному оклонению 
data.groupby('Regionname')['Lattitude'].std().sort_values()

In [ ]:
date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (date1 <= data['Date']) & (data['Date']<= date2)
data[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

In [ ]:
# Группировка Типов домов и Комнат по Цене
# (Unstack-Превращает выходные данные в формат DF) 
data.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

In [ ]:
data.pivot_table(
    values='Price', # Значения - Цена
    index='Rooms', # Индексы - Число комнат
    columns='Type', # Столбцы - Тип домов
    fill_value= 0 # Чем будут заполнены пропуски
)

In [ ]:
data.pivot_table(
    values='Price',
    index= 'Regionname',
    columns='Weekend',
    aggfunc='count')

In [ ]:
data.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    fill_value= 0,
    aggfunc= ['median', 'mean']
)

In [ ]:
data.pivot_table(
    values='Price',
    index=['Method', 'Type'],
    columns='Regionname',
    fill_value= 0,
    aggfunc= ['median', 'mean']
)

In [ ]:
pivot = data.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
mask = pivot['mean']['house'] < pivot['median']['house']
filter_pivot = pivot[mask]
filter_pivot
print(list(filter_pivot.index))

In [ ]:
pivot = data.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    fill_value= 0,
    aggfunc='median',
    )
max_price = pivot['unit'].max()
print(pivot[pivot['unit'] == max_price].index[0])